In [2]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adtk.detector import SeasonalAD
from adtk.data import validate_series

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["dadosVivamente"]
collection = db["postsAgregadoComRespostas"]

In [4]:
# Filtrando documentos com o atributo maior ou igual a 2
filtro = {"suicida": {"$eq": "3"}}
documentos = collection.find(filtro)

In [5]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))

In [6]:
# Visualizando o DataFrame
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,quantAmigos,postStory,postCreated_time,diaDaSemana,hora,minutos,diaDoMes,mes,ano,postMessage
0,593a0b9efff947001dcf8fe7,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,Gabriela Tedesco shared Chloe's photo.,2017-06-08 16:01:43,Quinta-feira,16,1,8,6,2017,NaN
1,593a0b9efff947001dcf8fe8,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,NaN,2017-06-08 15:56:50,Quinta-feira,15,56,8,6,2017,Mini pizza 😍😍😍
2,593a0b9efff947001dcf8fe9,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,NaN,2017-06-08 01:51:16,Quinta-feira,1,51,8,6,2017,Os pães de hj 😍😘😍
3,593a0b9efff947001dcf8fea,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,NaN,2017-06-08 01:12:01,Quinta-feira,1,12,8,6,2017,Amo um carinho da minha mãe 😍😍😍
4,593a0b9efff947001dcf8feb,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,Gabriela Tedesco shared Chloe's photo.,2017-06-07 23:41:27,Quarta-feira,23,41,7,6,2017,NaN


In [7]:
df['postCreated_time'] = pd.to_datetime(df['postCreated_time'])
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,quantAmigos,postStory,postCreated_time,diaDaSemana,hora,minutos,diaDoMes,mes,ano,postMessage
0,593a0b9efff947001dcf8fe7,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,Gabriela Tedesco shared Chloe's photo.,2017-06-08 16:01:43,Quinta-feira,16,1,8,6,2017,NaN
1,593a0b9efff947001dcf8fe8,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,NaN,2017-06-08 15:56:50,Quinta-feira,15,56,8,6,2017,Mini pizza 😍😍😍
2,593a0b9efff947001dcf8fe9,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,NaN,2017-06-08 01:51:16,Quinta-feira,1,51,8,6,2017,Os pães de hj 😍😘😍
3,593a0b9efff947001dcf8fea,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,NaN,2017-06-08 01:12:01,Quinta-feira,1,12,8,6,2017,Amo um carinho da minha mãe 😍😍😍
4,593a0b9efff947001dcf8feb,27,F,1870214326566563,2.0,2,1,1,3,2,...,532,Gabriela Tedesco shared Chloe's photo.,2017-06-07 23:41:27,Quarta-feira,23,41,7,6,2017,NaN


In [8]:
df['data'] = df['postCreated_time'].dt.to_period('M').dt.to_timestamp()
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,postStory,postCreated_time,diaDaSemana,hora,minutos,diaDoMes,mes,ano,postMessage,data
0,593a0b9efff947001dcf8fe7,27,F,1870214326566563,2.0,2,1,1,3,2,...,Gabriela Tedesco shared Chloe's photo.,2017-06-08 16:01:43,Quinta-feira,16,1,8,6,2017,NaN,2017-06-01
1,593a0b9efff947001dcf8fe8,27,F,1870214326566563,2.0,2,1,1,3,2,...,NaN,2017-06-08 15:56:50,Quinta-feira,15,56,8,6,2017,Mini pizza 😍😍😍,2017-06-01
2,593a0b9efff947001dcf8fe9,27,F,1870214326566563,2.0,2,1,1,3,2,...,NaN,2017-06-08 01:51:16,Quinta-feira,1,51,8,6,2017,Os pães de hj 😍😘😍,2017-06-01
3,593a0b9efff947001dcf8fea,27,F,1870214326566563,2.0,2,1,1,3,2,...,NaN,2017-06-08 01:12:01,Quinta-feira,1,12,8,6,2017,Amo um carinho da minha mãe 😍😍😍,2017-06-01
4,593a0b9efff947001dcf8feb,27,F,1870214326566563,2.0,2,1,1,3,2,...,Gabriela Tedesco shared Chloe's photo.,2017-06-07 23:41:27,Quarta-feira,23,41,7,6,2017,NaN,2017-06-01


In [9]:
df_grouped_data = df.groupby(['id_usuario', 'data']).size().reset_index(name='quantidade')
# Substituir o NaN por 0
df_grouped_data['quantidade'] = df_grouped_data['quantidade'].fillna(0)

In [10]:
print(df.dtypes)

_id                         object
idade                        int64
sexo                        object
id_usuario                  object
nivel                      float64
pessimismo                  object
tristeza                    object
fracasso                    object
prazer                      object
culpa                       object
punicao                     object
estima                      object
critica                     object
suicida                     object
choro                       object
agitacao                    object
interesse                   object
indecisao                   object
desvalorizacao              object
energia                     object
sono                        object
irritabilidade              object
apetite                     object
concentracao                object
fadiga                      object
int_sexo                    object
quantAmigos                 object
postStory                   object
postCreated_time    

In [11]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [12]:
print(df.dtypes)

_id                         object
idade                        int64
sexo                        object
id_usuario                  object
nivel                      float64
pessimismo                   int64
tristeza                     int64
fracasso                     int64
prazer                       int64
culpa                        int64
punicao                      int64
estima                       int64
critica                      int64
suicida                      int64
choro                        int64
agitacao                     int64
interesse                    int64
indecisao                    int64
desvalorizacao               int64
energia                      int64
sono                         int64
irritabilidade               int64
apetite                      int64
concentracao                 int64
fadiga                       int64
int_sexo                     int64
quantAmigos                  int64
postStory                   object
postCreated_time    

In [13]:
# Gerar o gráfico de anomalias de cada usuário
for usuario_id in df_grouped_data['id_usuario'].unique():
    df_usuario = df_grouped_data[df_grouped_data['id_usuario'] == usuario_id]

    # Salvar em csv
    # df_usuario.to_csv(f'./dados/{usuario_id}.csv', index=False)

    # Gerar o gráfico
    df_usuario.set_index('data', inplace=True)

    # Extrair a série temporal 'quantidade'
    s = df_usuario['quantidade']

    # Validar a série temporal
    s = validate_series(s)

    # Usar SeasonalAD para ver se detecta outras anomalias
    seasonal_ad = SeasonalAD()
    anomalies_seasonal = seasonal_ad.fit_detect(s)

    # Plotar os resultados do SeasonalAD
    plt.figure(figsize=(14, 7))
    sns.lineplot(x=s.index.astype(str), y=s.values, label='Quantidade de Posts')
    anomaly_points_seasonal = s[anomalies_seasonal == True]
    sns.scatterplot(x=anomaly_points_seasonal.index.astype(str), y=anomaly_points_seasonal.values, color='red', s=100,
                    label='Anomalias (SeasonalAD)')
    plt.title(f'Usuário {usuario_id} - Quantidade de Posts por Mês/Ano (SeasonalAD)')
    plt.xlabel('Mês/Ano')
    plt.ylabel('Quantidade de Posts')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.show()

RuntimeError: Series does not follow any known frequency (e.g. second, minute, hour, day, week, month, year, etc.